In [26]:
!pip install graphviz
!pip install transformers
!pip install bert
!pip install bert-tensorflow
!pip install keras
!pip install dask_ml

     |████████████████████████████████| 141 kB 9.0 MB/s eta 0:00:01


In [31]:
# Import relevent packages
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import pickle

# Import dask packages
import dask.dataframe as ddf
from math import nan
import panel as pn
import dask
import dask.dataframe as dd
from dask.delayed import delayed
import seaborn as sns
import matplotlib.pyplot as plt
from dask_ml.model_selection import train_test_split
import graphviz
from datascroller import scroll

from dask.diagnostics import ProgressBar
ProgressBar().register()

# text processing libraries
import re
import string
import nltk
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')
# from sklearn.model_selection import train_test_split

import json
import os
import sys
import tensorflow as tf

import transformers as ppb
from time import time
import io
import re
from csv import reader

import bert
# from bert import run_classifier
# from bert import optimization
from bert import tokenization
from transformers import RobertaTokenizer, TFRobertaModel

from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow import keras
#### if use tensorflow=2.0.0, then import tensorflow.keras.model_selection 
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.models import load_model

from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score


from datetime import datetime

## Install BERT and BERT Tokenizer from the HuggingFace Transformers library.

We can easily switch between variants of BERT by changing out which model we import from HuggingFace; the rest of the code just flows unchanged.

In [14]:
# For DistilBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.TFDistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.TFBertModel, ppb.BertTokenizer, 'bert-base-uncased')

## For BERT Large, use this:
# model_class, tokenizer_class, pretrained_weights = (ppb.AutoModelWithLMHead, ppb.AutoTokenizer, 'bert-large-uncased')
# from transformers import AutoTokenizer, AutoModelWithLMHead

# For ROBERTa base model, use this:
# model_class, tokenizer_class, pretrained_weights = (ppb.TFRobertaModel, ppb.RobertaTokenizer, 'roberta-base')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## Let's do an initial import on the sample dataset Padma created for HS4 codes 8712 and 8714

In [15]:
!ls /data/common/trade_data

 2019
 HS
 enigma
 import_data_2017_enhanced.parq
 import_data_2018_enhanced.parq
 import_data_2019_enhanced.parq
 import_data_2020_enhanced.parq
 linux-bsd.gif
'manifestDB - CMDT Codes directory.xlsx'
'manifestDB - Data Fields Description.xlsx'
'manifestDB - Main Coverage Ports Directory.xlsx'
'manifestDB - Packing Unit Description.xlsx'
 raw_import_20152016.parq
 raw_import_2017.parq
 raw_import_2018.parq
 raw_import_2019.parq
 raw_import_2020.parq
 us_customs_aug2019.parq
 w210_raw_data.ipynb


In [16]:
import_df = dd.read_parquet('/data/common/trade_data/2019/data_samples/sample_87128714.parq', engine='fastparquet', chunksize="100MB")

In [17]:
import_df.columns

Index(['System Identity Id', 'Estimate Arrival Date', 'Actual Arrival Date',
       'Bill of Lading', 'Master Bill of Lading', 'Bill Type Code',
       'Carrier SASC Code', 'Vessel Country Code', 'Vessel Code',
       'Vessel Name', 'Voyage', 'Inbond Type', 'Manifest No',
       'Mode of Transportation', 'Loading Port', 'Last Vist Foreign Port',
       'US Clearing District', 'Unloading Port', 'Place of Receipt', 'Country',
       'Country Sure Level', 'Weight in KG', 'Weight', 'Weight Unit', 'TEU',
       'Quantity', 'Quantity Unit', 'Measure in CM', 'Measure', 'Measure Unit',
       'Container Id', 'Container Size', 'Container Type',
       'Container Desc Code', 'Container Load Status',
       'Container Type of Service', 'Shipper Name', 'Shipper Address ',
       'Raw Shipper Name', 'Raw Shipper Addr1', 'Raw Shipper Addr2',
       'Raw Shipper Addr3', 'Raw Shipper Addr4', 'Raw Shipper Addr Others',
       'Consignee Name', 'Consignee Address ', 'Raw Consignee Name',
       'Raw Con

In [18]:
# Don't truncate text fields in the display
pd.set_option("display.max_colwidth", -1)

import_df.head(2)

[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  2.3s
[########################################] | 100% Completed |  2.4s
[########################################] | 100% Completed |  2.4s
[########################################] | 100% Completed |  2.5s
[########################################] | 100% Completed |  2.5s


,System Identity Id,Estimate Arrival Date,Actual Arrival Date,Bill of Lading,Master Bill of Lading,Bill Type Code,Carrier SASC Code,Vessel Country Code,Vessel Code,Vessel Name,...,Product Desc,Marks & Numbers,HS Code Sure Level,CIF,Indicator of true supplier,Indicator of true buyer,END,HS Code,HS_Code,Merged_Description
0,6003201907090000254809,20190705,20190705,CHSLTPE19060165,EGLV003901609611,H,"CHSL, CHISLEY MOTOR COACHES",PA,9306990,OOCL VANCOUVER,...,"ELLIPTICAL TRAINER,TREADMILL, ESCALATE<br/>",NO MARKS<br/>,5,0.0,Y,Y,END,871200,871200,"Bicycles and other cycles (including delivery tricycles), not motorized ;Bicycles having both wheels not exceeding 63.5 cm in diameter;Having both wheels not exceeding 50 cm in diameter;Having both wheels exceeding 50 cm but not exceeding 55 cm in diameter;Having both wheels exceeding 55 cm but not exceeding 63.5 cm in diameter;If weighing less than 16.3 kg complete without accessories and not designed for use with tires having a cross-sectional diameter exceeding 4.13 cm;Bicycles having both wheels exceeding 63.5 cm in diameter ;Other;Bicycles having a front wheel exceeding 55 cm but not exceeding 63.5 cm in diameter and a rear wheel exceeding 63.5 cm in diameter, weighing less than 16.3 kg complete without accessories and not designed for use with tires having a cross-sectional diameter exceeding 4.13 cm, valued $200 or more each;Other bicycles;Other cycles"
1,6003201901040000381477,20181230,20190103,AMAWSHNGBA807064,APLUNPFB000624,H,AMAW,CN,APL BARCELONA,APL BARCELONA,...,SCH 830 TREADMILL<br/>,AS ADDRESSED<br/>,5,0.0,Y,Y,END,871200,871200,"Bicycles and other cycles (including delivery tricycles), not motorized ;Bicycles having both wheels not exceeding 63.5 cm in diameter;Having both wheels not exceeding 50 cm in diameter;Having both wheels exceeding 50 cm but not exceeding 55 cm in diameter;Having both wheels exceeding 55 cm but not exceeding 63.5 cm in diameter;If weighing less than 16.3 kg complete without accessories and not designed for use with tires having a cross-sectional diameter exceeding 4.13 cm;Bicycles having both wheels exceeding 63.5 cm in diameter ;Other;Bicycles having a front wheel exceeding 55 cm but not exceeding 63.5 cm in diameter and a rear wheel exceeding 63.5 cm in diameter, weighing less than 16.3 kg complete without accessories and not designed for use with tires having a cross-sectional diameter exceeding 4.13 cm, valued $200 or more each;Other bicycles;Other cycles"


In [19]:
len(import_df.index)

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  0.5s
[########################################] | 100% Completed |  0.6s
[########################################] | 100% Completed |  0.6s


7269

Let's tokenize the description fields.

In [23]:
def tokenize(text):
    '''
    Tokenize text
    '''
    tokens = nltk.word_tokenize(text)
    
    return list(
        filter(lambda word: word.isalnum(), tokens)
    )

stop_words = stopwords.words("english")
# ## Add some common words from text
# stop_words.extend(["from","subject","summary","keywords","article"])
def remove_stopwords(words):
    '''
    Remove stop words from the list of words
    '''
    
    filtered = filter(lambda word: word not in stop_words, words)
    
    return list(filtered)

### Create train, dev, and test sets

In [42]:
# 20% to test, 20% to dev and 60% to train

# Set aside a test set
train_df, test_df = train_test_split(import_df, test_size=0.2)

# Split into train and dev sets
train_df, dev_df = train_test_split(train_df, test_size=0.25)

print('Size of train set is', len(train_df.index))
print('Size of dev set is', len(dev_df.index))
print('Size of test set is', len(test_df.index))

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  0.5s
[########################################] | 100% Completed |  0.6s
[########################################] | 100% Completed |  0.7s
[########################################] | 100% Completed |  0.8s
[########################################] | 100% Completed |  0.9s
Size of train set is  4327
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[####################

In [48]:
# convert the dataframes back to pandas
train_df_pd = train_df.compute()
dev_df_pd = dev_df.compute()
test_df_pd = test_df.compute()

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  0.5s
[########################################] | 100% Completed |  0.6s
[########################################] | 100% Completed |  0.7s
[########################################] | 100% Completed |  0.8s
[########################################] | 100% Completed |  0.9s
[########################################] | 100% Completed |  1.0s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100

## BERT model

In [45]:
def pre_process(data, max_length):
    """
    Function takes inputs:
    - data in the form of a dataframe with a column named 'tweet'
    - max length
    and produces as output the input data BERT requires as an array consisting of:
    - Sentence IDs padded to the max length
    - BERT Masks that tell BERT which of the Sentence IDs are 0 and should be ignored
    - SequenceIDs which are all 0 for our classification task
    """
    # Tokenize each tweet and add the special beginning/end tokens
    tokenized = data['Product Desc'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_length, truncation=True)))
      
    # Create the padding based on the max length so all are same shape
    bertSentenceIDs = np.array([i + [0]*(max_length-len(i)) for i in tokenized.values])
    
    # Create the attention mask so BERT knows which contain values and which are 0s that should be ignored
    bertMasks = np.where(bertSentenceIDs != 0, 1, 0)

    # Create the BERT sequence IDs. In this case they are all 0 since it's the same sentence input.
    bertSequenceIDs = np.array([np.zeros(max_length) for i in tokenized.values], dtype=int)
    
    # Create and return the data array containing both the padded and the attention mask
    X_data = np.array([bertSentenceIDs, bertMasks, bertSequenceIDs])

    # Also look at the vocabulary size in the tokenizer

    return X_data

In the past we have run into memory issues depending on the length of the input, so we set up a variable to truncate the tokens being input for each record.

In [58]:
max_length = 150

X_train = pre_process(train_df_pd, max_length)
X_dev = pre_process(dev_df_pd, max_length)

train_labels = np.array(train_df_pd['HS_Code']).astype(int)
dev_labels = np.array(dev_df_pd['HS_Code']).astype(int)

In [59]:
# Let's make sure we end up with the shapes we expect.
print('X_train shape:', X_train.shape)
print('X_train labels shape:', train_labels.shape)
print()

print('X_dev shape:', X_dev.shape)
print('X_dev labels shape:', dev_labels.shape)
print()


X_train shape: (3, 4327, 150)
X_train labels shape: (4327,)

X_dev shape: (3, 1509, 150)
X_dev labels shape: (1509,)



In [60]:
X_train[1]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [61]:
train_labels[1]

871200

### Construct the model

In [64]:
def sent_model(max_input_length, train_layers, optimizer):
    """
    UPDATE doc string
    Implementation of multi classification model
    
    variables:
        max_input_length: number of tokens (max_length + 1)
        train_layers: number of layers to be retrained
        optimizer: optimizer to be used
    
    returns: model
    """
    
    in_id = tf.keras.layers.Input(shape=(max_input_length,), dtype='int32', name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_input_length,), dtype='int32', name="attention_masks")
    in_segment = tf.keras.layers.Input(shape=(max_input_length,), dtype='int32', name="segment_ids")
    
    bert_inputs = [in_id, in_mask, in_segment]
    
    
    # Note: Bert layer from Hugging Face returns two values: sequence ouput, and pooled output. Here, we only want
    # the former. (See https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel) 
    
    bert_layer = model_class.from_pretrained(pretrained_weights)
    
    
    # Freeze layers, i.e. only train number of layers specified, starting from the top
    
    retrain_layers = []
    
    for retrain_layer_number in range(train_layers):
        
        layer_code = '_' + str(11 - retrain_layer_number)
        retrain_layers.append(layer_code)
    
    for w in bert_layer.weights:
        if not any([x in w.name for x in retrain_layers]):
            w._trainable = False
            
    # End of freezing section
    
    # bert_layer(bert_inputs) generates two outputs. 
    # Select this to use the average of the non-CLS tokens to make the prediction
    # bert_sequence = tf.math.reduce_mean(bert_layer(bert_inputs)[0], axis=1) # take the average of all the non-CLS tokens
    
    # Select this to use the CLS token to make the prediction
    bert_sequence = bert_layer(bert_inputs)[1]
    
    print('Let us check the shape of the BERT layer output:', bert_sequence)
    
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_sequence)
    
    dropped = tf.keras.layers.Dropout(rate=0.1)(dense)
    
    pred = tf.keras.layers.Dense(1, activation='sigmoid', name='target')(dropped) 
     
    print('pred: ', pred)
    
   
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=optimizer, metrics=['accuracy'])  

    
    model.summary()
    
    return model

In [65]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# Set hyperparameters
adam_customized = tf.keras.optimizers.Adam(lr=0.0005, beta_1=0.91, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False)
epochs = 10
batch_size = 128
train_layers = 2

# ###Convert the labels into 1 and 0 for binary_crossentropy

# new_train_labels = np.where(train_labels_b=="TIN", 1, 0)

# new_dev_labels = np.where(dev_labels_b=="TIN", 1, 0)


training_labels = train_labels


training_data = X_train

dev_data = X_dev


model = sent_model(max_length, train_layers=train_layers, optimizer = adam_customized)



# Set parameters for early stopping
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        min_delta=0.001,
                                                        patience=5,
                                                        verbose=0, mode='auto')
# Capture the history of the model fitting
history = model.fit(
    x = {"input_ids": training_data[0],
         "attention_masks": training_data[1],
         "segment_ids": training_data[2]}, y = {"target": training_labels},
    validation_data=({"input_ids": dev_data[0],
         "attention_masks": dev_data[1],
         "segment_ids": dev_data[2]}, {"target": dev_labels}),
    verbose=2,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping_callback]
)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Let us check the shape of the BERT layer output: KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model_4/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model_4'")
pred:  KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='target/Sigmoid:0', description="created by layer 'target'")
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 150)]        0                                            
__________________________________________________________________________________________________
segmen

InvalidArgumentError:  Received a label value of 871499 which is outside the valid range of [0, 1).  Label values: 871410 871200 871493 871493 871420 871496 871491 871420 871496 871410 871200 871420 871492 871491 871494 871200 871499 871493 871493 871493 871410 871200 871200 871494 871492 871420 871493 871494 871420 871410 871420 871493 871493 871492 871499 871200 871492 871493 871493 871492 871493 871495 871420 871492 871494 871492 871499 871410 871200 871410 871492 871493 871200 871496 871200 871420 871499 871410 871496 871410 871200 871200 871200 871493 871410 871493 871493 871420 871493 871420 871200 871499 871493 871420 871420 871200 871420 871492 871493 871492 871200 871200 871410 871499 871492 871494 871420 871499 871492 871494 871496 871420 871499 871493 871491 871494 871200 871200 871492 871420 871494 871494 871492 871420 871200 871410 871491 871494 871496 871492 871493 871420 871491 871496 871420 871492 871494 871200 871200 871496 871493 871200 871492 871499 871499 871420 871